In [99]:
import numpy as np
from random import randint
from scipy.optimize import fsolve
from utils.demand_functions import LogitDemand
demandfn = LogitDemand({
    "a0":0,
    "cost": 1,
    "mu":0.25,
    "a": 2
})

def foc(p):
    """Compute first order condition"""
    d = demandfn.demand(p)
    zero = 1 - (p - demandfn.cost) * (1 - d) / demandfn.mu
    return np.squeeze(zero)

def foc_monopoly(p):
    """Compute first order condition of a monopolist"""
    d = demandfn.demand(p)
    d1 = np.flip(d)
    p1 = np.flip(p)
    zero = 1 - (p - demandfn.cost) * (1 - d) / demandfn.mu + (p1 - demandfn.cost) * d1 / demandfn.mu
    return np.squeeze(zero)
def compute_p_competitive_monopoly():
    """Computes competitive and monopoly prices"""
    p0 = np.ones((1, 2)) * 1 * demandfn.cost
    p_competitive = fsolve(foc, p0)
    p_monopoly = fsolve(foc_monopoly, p0)
    return p_competitive, p_monopoly
compute_p_competitive_monopoly()

(array([1.47292666, 1.47292666]), array([1.92498092, 1.92498092]))

In [1]:
from gymnasium.spaces.discrete import Discrete

In [141]:
from environments.SimpleOligopolyEnv import SimpleOligopolyEnv

env_conf = {
    "max_price" : 10,
    "min_price" : 0,
    "num_sellers" : 2,
    "memory_size" : 5,
    "action_type" : "disc",
    "disc_action_size" : 6,
    "max_steps" : 10,
    "agent_ids" : ["agent0", "agent1"],
    "market" :{
    "demand_type" : "Logit",
        "logit" : {
        "a0" : 0,
        "cost" : 1,
        "mu" : 0.25,
        "a" : 2,
        },
        "linear" : None
    },
    "action_space": Discrete(6)
}
env = SimpleOligopolyEnv(seed = 0 , config=env_conf)
env.reset()

({'agent0': array([1.71116908, 1.88788407, 1.59788827, 1.92844323, 1.5161056 ,
         1.89953563, 1.60915439, 1.7088085 , 1.56447591, 1.96775915]),
  'agent1': array([1.71116908, 1.88788407, 1.59788827, 1.92844323, 1.5161056 ,
         1.89953563, 1.60915439, 1.7088085 , 1.56447591, 1.96775915])},
 {'agent0': {}, 'agent1': {}})

In [180]:
env.step({"agent0":5, "agent1":4})

({'agent0': array([1.56447591, 1.96775915, 1.53621426, 1.42772123, 1.97018634,
         1.42772123, 1.97018634, 1.97018634, 1.86169332, 1.97018634]),
  'agent1': array([1.56447591, 1.96775915, 1.53621426, 1.42772123, 1.97018634,
         1.42772123, 1.97018634, 1.97018634, 1.86169332, 1.97018634])},
 {'agent0': 0.28277424, 'agent1': 0.38762257},
 {'agent0': False, 'agent1': False, '__all__': False},
 {'agent0': False, 'agent1': False, '__all__': False},
 {'agent0': {}, 'agent1': {}})

In [182]:
env.possible_actions

array([1.42772123, 1.53621426, 1.64470728, 1.7532003 , 1.86169332,
       1.97018634])

In [183]:
def br_profits(prices, a, a0, mu, cost, index, p_index):
    prices[index]=p_index
    utility = np.exp((a - prices) / mu)
    demand = utility / (np.sum(utility) + np.exp(a0 / mu))
    profits = (prices - cost) * demand
    return profits[index]

In [187]:
def obj_func(p_index):
    prices = env.get_last_prices()
    prices = prices.copy()
    return -br_profits(prices, 2,0,0.25,1,1,p_index)

In [188]:
from scipy.optimize import minimize
pj = minimize(obj_func, [1.0], method='L-BFGS-B')

/tmp/ipykernel_32565/2650556334.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  prices[index]=p_index


In [189]:
pj.x[0]

1.6381392964675354